In [1]:
import numpy as np
import sympy as sp
from scipy.optimize import fsolve
t, y = sp.symbols("t y", real=True)

In [56]:
def f(y):
    return y**(1/2)

def sol(y):
    return (1./4.)*(4. + 4.*y + y**2)
## y = y_i+1
def F(y, y_i, h):
    return y - y_i - h*f(y)

def dF(y):
    return 1 - h*df(y)

def newton(t, y_0, h, maxiter = 100, tol = 1e-15):
    y = y_0
    for i in range(maxiter):
        w = F(t, y, y_0, h)/dF(t, y, h)
        y -= w
        if F(t, y, y_0, h) < tol:
            print "tol"
            break
    return y

def solver(f, h, y_0, t_0, t_f):
    n = int((t_f-t_0)/h)
    #y = np.zeros((len(y_0), n+1))
    y = np.empty((n+1,))
    trange = np.linspace(t_0, t_f, n+1)
    y[0] = y_0
    for i in range(1,n+1):
        y[i] = fsolve(F, x0 = y[i-1], args = (y[i-1], h))
    return y, trange
    

In [57]:
y, t = solver(f, 0.00002, 1, 0, 1)
print abs(sol(t) - y)

[  0.00000000e+00   5.00012032e-10   1.20003230e-09 ...,   2.49999999e-01
   2.50010000e-01   2.50020000e-01]


In [58]:
sol(t)

array([ 1.     ,  1.00002,  1.00004, ...,  2.24994,  2.24997,  2.25   ])

In [59]:
y

array([ 1.     ,  1.00002,  1.00004, ...,  1.99994,  1.99996,  1.99998])